In [1]:
# Import libraries
from gurobipy import*

import math
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

In [2]:
# File path 
FILE = "./DaSilvaUrrutia/n200w100.001.txt"

In [3]:
# Default params
SUPPORTED_FORMAT = ['NUM', 'X', 'Y', 'DEMAND', 'READYTIME', 'DUEDATE', 'SERVICE']
MINUTES = 60
HOURS = 3600
OFFSET_TIMES = 8*HOURS

COLUMNS_OPS = {'NUM': lambda x: float(x),
               'X': lambda x: float(x),
               'Y': lambda x: float(x),
               'DEMAND': lambda x: 1,
               'READYTIME': lambda x: float(x)+OFFSET_TIMES,
               'DUEDATE': lambda x: float(x)+OFFSET_TIMES,
               'SERVICE': lambda x: 30*MINUTES
              }

AGENTS = 5

TIME_PER_DISTANCE = 1

WORKING_TIME_RANGE = (OFFSET_TIMES, OFFSET_TIMES + 8*HOURS)

LUNCH_BREAK_RANGE = (12*HOURS, 13.5*HOURS)
LUNCH_BREAK_TIME = 30*MINUTES

OFFICE_NUM = 0
OFFICE_X = .0
OFFICE_Y = .0
OFFICE_READYTIME = WORKING_TIME_RANGE[0]
OFFICE_DUEDATE = WORKING_TIME_RANGE[1]
OFFICE_SERVICE = 1*HOURS

In [4]:
def read_input_tsptw(filename):
    """This function is used to convert input file to usable data"""
    nb_nodes = 0
    
    data_dict = dict()
    
    nodes_x = list()
    nodes_y = list()
       
    # Open file and read lines 
    with open(filename, "r") as file:
        # Initialize columns in empty dict
        columns = file.readline().replace("#","").split()
        if columns != SUPPORTED_FORMAT:
            print("ERROR! Format not supported.")
            return 
        
        # Add office to data
        data_dict.update({OFFICE_NUM: {'X': OFFICE_X, 
                                       'Y': OFFICE_Y, 
                                       'DEMAND': AGENTS,
                                       'READYTIME': OFFICE_READYTIME,
                                       'DUEDATE': OFFICE_DUEDATE,
                                       'SERVICE': OFFICE_SERVICE,}})
        # Add office to nodes
        nodes_x.append(OFFICE_X)
        nodes_y.append(OFFICE_Y)
            
        # For each data line
        for line in file.readlines():
            node_dict = {k: COLUMNS_OPS[k](val) for k, val in zip(columns, line.split())}
                
            # Get id
            node_id = node_dict.pop('NUM')
            # Insert new node in data dict
            data_dict.update({int(node_id): node_dict})            
            # Get nodes positions
            nodes_x.append(float(line.split()[columns.index('X')]))
            nodes_y.append(float(line.split()[columns.index('Y')]))

    # Get distance matrix
    distance_matrix = compute_distance_matrix(nodes_x, nodes_y)
    
    return (data_dict, distance_matrix, dict(enumerate(zip(nodes_x, nodes_y))))


def compute_distance_matrix(nodes_x, nodes_y):
    """This function is used to compute the distance matrix"""
    nb_customers = len(nodes_x)
    distance_matrix = [[None for i in range(nb_customers)] for j in range(nb_customers)]
    for i in range(nb_customers):
        distance_matrix[i][i] = 0
        for j in range(nb_customers):
            dist = compute_dist(nodes_x[i], nodes_x[j], nodes_y[i], nodes_y[j])
            distance_matrix[i][j] = dist
            distance_matrix[j][i] = dist
    return distance_matrix


def compute_dist(xi, xj, yi, yj):
    """This function is used to compute euclidean distance"""
    exact_dist = math.sqrt(math.pow(xi - xj, 2) + math.pow(yi - yj, 2))
    return int(math.floor(exact_dist + 0.5)) * TIME_PER_DISTANCE

In [5]:
# Getting parameters
data_dict, distance_matrix, positions = read_input_tsptw(FILE)

In [6]:
# DEBUG RESTRICTIONS

data_dict = {k: v for k,v in data_dict.items() if k < 20}
distance_matrix = distance_matrix[:20][:20]


### Define function to plot the graph

In [7]:
def plot_model(model, positions):
    """This function is used to plot graph"""
    # Create figure
    plt.figure(figsize=(16,9))
    G = nx.Graph()
    # Add nodes to graph
    G.add_nodes_from(positions.keys())
    
    # Set labels dict
    node_labels = dict()
    # Set nodes positions
    for n, p in positions.items():
        G.nodes[n]['pos'] = p
        node_labels[n] = n

    # Define edges colors
    edge_colors = [plt.cm.tab20.colors[i] for i in model.keys()]
    # Add edges
    for t, l in model.items():
        for i in range(1,len(l['trip'])):
            G.add_edge(l['trip'][i-1], l['trip'][i], 
                       color=edge_colors[t], 
                       alpha=0.5, 
                       weight=4)
    # Set edges parameters 
    edges = G.edges()
    colors = [G[u][v]['color'] for u,v in edges]
    weights = [G[u][v]['weight'] for u,v in edges]
    alphas = [G[u][v]['alpha'] for u,v in edges]

    # Draw Nodes
    nx.draw_networkx_nodes(G, positions,
                           node_size=200,
                           node_color='c')
    # Draw Labels
    nx.draw_networkx_labels(G, positions, 
                            node_labels, 
                            font_size=10, 
                            font_color='k')
    # Draw Edges 
    nx.draw_networkx_edges(G, positions, 
                           alpha=0.5, 
                           width=weights, 
                           edge_color=colors)

    # Show graph
    plt.show()

In [8]:
# Create model
mod = Model("TSPTW")     

Using license file c:\gurobi903\gurobi.lic
Academic license - for non-commercial use only


In [9]:
# Add variables
# Agent serve client
x = mod.addVars({(i,a): 0 for i in range(len(data_dict)*2)
                          for a in range(AGENTS)},
               name="x",
               vtype=GRB.BINARY)

#  Agent trip
y = mod.addVars({(i,j,a): 0 for i in range(len(data_dict)*2) 
                            for j in range(len(data_dict)*2) 
                            for a in range(AGENTS)}, 
                name="y", 
                vtype=GRB.BINARY)

# Serve Order
z = mod.addVars({(i,a): 0 for i in range(len(data_dict)*2) 
                          for a in range(AGENTS)},
               name="z",
               vtype=GRB.INTEGER)
# #  Wait time
# y = mod.addVars({(i,a): 0 for i in range(len(data_dict)) 
#                           for a in range(AGENTS)}, 
#                 name="y", 
#                 vtype=GRB.INTEGER)

# #  Minutes in office
# z = mod.addVars({(i,j,a): 0 for i in range(len(data_dict)) 
#                             for j in range(len(data_dict)) 
#                             for a in range(AGENTS)}, 
#                 name="z", 
#                 vtype=GRB.INTEGER)

# # Service time of client i
# s =  mod.addVars({(i,a): 0 for i in range(len(data_dict)) 
#                            for a in range(AGENTS)}, 
#                  name="s", 
#                  vtype=GRB.INTEGER)

In [10]:
all_pos = [pos for pos in list(range(len(data_dict)*2)) if pos not in [len(data_dict),]]
clients_pos = list(range(1,len(data_dict)))
reachable_pos = [pos for pos in list(range(len(data_dict)*2)) if pos not in [0,len(data_dict),]]
office_pos = [pos for pos in list(range(len(data_dict)+1, len(data_dict)*2))]

In [12]:
_= mod.addConstrs((quicksum(x[i,a]
                            for a in range(AGENTS)) == 1.
                  for i in clients_pos),
                  name="ServeAll")

_= mod.addConstrs((quicksum(y[i,j,a]
                            for i in all_pos if i != j
                           ) <= x[j,a] 
                  for a in range(AGENTS)
                  for j in reachable_pos),
                  name="ServeOnceA")

_= mod.addConstrs((quicksum(y[i,j,a] 
                            for j in reachable_pos if i != j
                           ) <= x[i,a]
                  for a in range(AGENTS)
                  for i in all_pos),
                  name="ServeOnceB")

_= mod.addConstrs((z[i,a] == x[i,a]+quicksum(z[j,a]*y[j,i,a] for j in all_pos if i!=j) 
                  for a in range(AGENTS)
                  for i in all_pos),
                  name="ServeOrder")

_= mod.addConstrs((quicksum(z[i,a] for i in all_pos) == quicksum(x[i,a] for i in all_pos)*
                                                           (quicksum(x[i,a] for i in all_pos)+1)/2
                  for a in range(AGENTS)),
                  name="CheckServeOrder")

_= mod.addConstrs((quicksum(x[i,a] for i in [0]+office_pos) >= 1 
                            for a in range(AGENTS)),
                  name="ServeOffice")

_= mod.addConstrs((x[len(data_dict),a] == 0  
                            for a in range(AGENTS)),
                  name="NoOfficeFromOfficeA")

_= mod.addConstrs((y[i,j,a] == 0  
                            for i in office_pos + [0]
                            for j in office_pos
                            for a in range(AGENTS)),
                  name="NoOfficeFromOfficeB")

_= mod.addConstrs((y[i,0,a] == 0  
                            for i in all_pos
                            for a in range(AGENTS)),
                  name="NoOfficeFromOfficeC")

_= mod.addConstrs((quicksum(y[i,h,a]+y[h,j,a] 
                            for j in clients_pos
                            for i in clients_pos) == 2*x[h,a]
                   for h in office_pos
                  for a in range(AGENTS)),
                  name="ServeOfficeA")

_= mod.addConstrs((x[i,a] >= x[i+len(data_dict),a]
                            for i in clients_pos
                  for a in range(AGENTS)),
                  name="BackToOfficeAfterClient")

# _= mod.addConstrs((quicksum(y[i,h,a]
#                             for i in clients_pos) >= 
#                    quicksum(y[h,j,a] 
#                             for j in clients_pos)
#                    for h in office_pos
#                   for a in range(AGENTS)),
#                   name="NotStartOnDestinationOffice")

# _= mod.addConstrs((quicksum(y[i,j,a] * distance_matrix[i][j]
#                            for i in range(len(data_dict))
#                            for j in range(len(data_dict))) +
# #                    quicksum(x[i,j,a] * y[j,a]
# #                            for i in range(len(data_dict))
# #                            for j in range(len(data_dict))) +
#                    quicksum(y[i,j,a] * data_dict[j]['SERVICE']
#                            for i in range(len(data_dict))
#                            for j in range(len(data_dict))) +
# #                    quicksum(x[i,0,a] * z[i,0,a]
# #                            for i in range(len(data_dict))) +
#                    WORKING_TIME_RANGE[0] <= WORKING_TIME_RANGE[1]
#                    for a in range(AGENTS)),
#                   name="MaxHours")

# _= mod.addConstrs((quicksum(x[i,i,a]
#                            for i in range(len(data_dict))) == 0
#                    for a in range(AGENTS)),
#                   name="TripToAnother")

# RANGE_MAX =max([data_dict[i]['DUEDATE'] + distance_matrix[i][j] - data_dict[j]['READYTIME']
#                        for i in range(len(data_dict))
#                        for j in range(len(data_dict))])

# _= mod.addConstrs((s[i,a] + distance_matrix[i][j] - RANGE_MAX * (1 - x[i,j,a]) + y[j,a] <= s[j,a]
#                    for i in range(len(data_dict))
#                    for j in range(len(data_dict))
#                    for a in range(AGENTS)),
#                   name="RouteInTime")

# _= mod.addConstrs((data_dict[i]['READYTIME'] <= s[i,a] + y[i,a]
#                    for i in range(len(data_dict))
#                    for a in range(AGENTS)),
#                   name="ServeAfterTWStart")

# _= mod.addConstrs((data_dict[i]['DUEDATE'] >= s[i,a]
#                    for i in range(len(data_dict))
#                    for a in range(AGENTS)),
#                   name="ServeBeforeTWEnd")

# _= mod.addConstrs((data_dict[j]['READYTIME'] - x[i,j,a] * s[j,a] == y[j,a]
#                    for i in range(len(data_dict))
#                    for j in range(len(data_dict))
#                    for a in range(AGENTS)),
#                   name="WaitForClient")

In [13]:
COST_WAIT = 2
# Add objective function
mod.setObjective(quicksum(y[i,j,a]*distance_matrix[i if i < len(data_dict) else 0][j if j < len(data_dict) else 0] # + y[j,a] * COST_WAIT
                                                         for i in all_pos
                                                         for j in all_pos
                                                         for a in range(AGENTS)), 
                 GRB.MINIMIZE)
    
# Optimize the model
#mod.params.Method=1
mod.params.TimeLimit=50
mod.optimize()

Changed value of parameter TimeLimit to 50.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 2699 rows, 8400 columns and 21015 nonzeros
Model fingerprint: 0xee628d39
Model has 200 quadratic constraints
Variable types: 0 continuous, 8400 integer (8200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  QMatrix range    [5e-01, 1e+00]
  QLMatrix range   [5e-01, 1e+00]
  Objective range  [4e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2485 rows and 6405 columns
Presolve time: 0.02s
Presolved: 8009 rows, 4560 columns, 24890 nonzeros
Variable types: 0 continuous, 4560 integer (2755 binary)

Root relaxation: objective 3.636364e-01, 3672 iterations, 0.22 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.36364    0   96       

In [14]:
# mod.computeIIS()

In [15]:
from copy import deepcopy
def domino(otrip):
    trip = deepcopy(otrip)
    sorted_trip = list()
    while len(trip):
        old = deepcopy(trip)
        # Find start
        for x in trip:
            not_start = False
            for y in trip:
                if x[0] == y[1]:
                    not_start = True
                    break
            if not not_start:
                sorted_trip.append(trip.pop(trip.index(x)))
                break
        # Continue dominos trip
        next_found = True
        while next_found:
            next_found = False
            for x in trip:
                if sorted_trip[-1][1] == x[0]:
                    sorted_trip.append(trip.pop(trip.index(x)))
                    next_found = True
                    break
                    
        if trip == old:
            for t in trip:
                sorted_trip.append(trip.pop(trip.index(x)))
    return sorted_trip

for a in range(AGENTS):
    agent_trip = list()
    for i in range(len(data_dict)*2):
        for j in range(len(data_dict)*2):
            if y[i,j,a].X:
                agent_trip.append((i if i < len(data_dict) else i,j if j < len(data_dict) else j))
                
    # Find start
    print(f"Agent {a}: {agent_trip}")

Agent 0: [(0, 14), (14, 12)]
Agent 1: [(0, 8), (4, 16), (8, 11), (11, 4)]
Agent 2: [(0, 7), (7, 6)]
Agent 3: [(0, 13), (5, 10), (10, 18), (13, 19), (18, 9), (19, 5)]
Agent 4: [(0, 2), (1, 15), (2, 3), (3, 17), (17, 1)]


In [22]:
# ORDINE DI VISITA (ordine di visita, cliente visitato)
for a in range(AGENTS):
    agent_clients = list()
    for i in all_pos:
        if z[i,a].X:
            agent_clients.append((z[i,a].X, i))
    # Find start
    print(f"Agent {a}: {sorted(agent_clients, key=lambda tup: tup[0])}")

Agent 0: [(1.0, 0), (2.0, 14), (3.0, 12)]
Agent 1: [(1.0, 0), (2.0, 8), (3.0, 11), (4.0, 4), (5.0, 16)]
Agent 2: [(1.0, 0), (2.0, 7), (3.0, 6)]
Agent 3: [(1.0, 0), (2.0, 13), (3.0, 19), (4.0, 5), (5.0, 10), (6.0, 18), (7.0, 9)]
Agent 4: [(1.0, 0), (2.0, 2), (3.0, 3), (4.0, 17), (5.0, 1), (6.0, 15)]


In [16]:
for a in range(AGENTS):
    agent_clients = list()
    for i in range(len(data_dict)*2):
        if x[i,a].X:
            agent_clients.append(i)
    # Find start
    print(f"Agent {a}: {agent_clients}")

Agent 0: [0, 12, 14]
Agent 1: [0, 4, 8, 11, 16]
Agent 2: [0, 6, 7]
Agent 3: [0, 5, 9, 10, 13, 18, 19]
Agent 4: [0, 1, 2, 3, 15, 17]
